# Environment Establishment
- Virtual Environment (venv.txt)

# Arrange the data to be used
- create resources folder inside Machine_Learning
- create Actual folder inside resources
- create 2 folder namely test and train in the Actual folder
- each of the 2 folder should have same set of folder which is (exportable, marketable, reject)
>> Note: the train and test folder should have excatly same named, count and syntax of named folder
>> Note: folder name should note have space if you want to change it
>> folder will serve as the name of the class

# Use the code below to create, run, test and save the model

# If you want to check the data/graph of learning of the model you can use the tensorboard_command.txt


In [1]:
# import Libraries

## Tensorflow one of the library that can build ML
import tensorflow as tf
## helper library which is tailored to code less
import helper as helper
## this will create a web ready model to be used for website
import tensorflowjs as tfjs
## use for re-initializing the custom library
import sys

## this object declaration of the class inside the helper library
_helper = helper.helper()

In [2]:
## every library declared here is to lessen the code sequence for the preparation of data, ML building and testing
from keras import Sequential, Model
from keras.utils import plot_model as pltmdl
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Input
from keras.layers import GlobalAveragePooling2D, RandomFlip, RandomCrop
from keras.layers import RandomRotation, RandomZoom, RandomHeight, RandomWidth
from keras.layers import Rescaling
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator as IDG
from keras.utils import image_dataset_from_directory

In [ ]:
## use to reload the custom library when there is change in the library
from importlib import reload
reload(sys.modules["helper"])

In [18]:
### The try here catches any error that will occur and report back the error and also with notification integrated
try:
  ## train_test_dir_setter is used to set the data, so called data preparations to be used by ML 
    ## the function can inspected inside the helper.py
  ## train_test_dir_setter, requires
    ## 1. test_var, train_var, class_var if you have the folder test and train only
    ## 2. test_var, train_var, eval_var, class_var if you have the folder test, train and eval
  ## for inputs; Requirements will be the directory of the file to be used. and also it has other more inputs just hover over train_test_dir_setter
  ## if the code is success it will display below the count of files found for test, train, eval and also the class names
  test_data, train_data, CLASS_NAMES = _helper.data_preparations.train_test_dir_setter("./resources/Actual/",IMG_SIZE = (500, 500),CLASS_MODE = "categorical")
except Exception as e:
  print(f"Error: {e}")
  _helper.notifications.error()

Test Data: 
Found 90 files belonging to 3 classes.
Train Data: 
Found 920 files belonging to 3 classes.
Class Names: ['exportable', 'marketable', 'reject']


In [19]:
try:
  ## the checkpoint_callback will produce call back function for the ML to store the best state it was when trained
  ## this produce two variable the checkpoint function and the checkpoint saving the checkpoint directory
  ## this require where to save (dir), file name (name)
  ## the val_loss inside and the other true are not required and for further information just hover over checkpoint_callback
  checkpoint, ckpt_dir = _helper.callbacks.checkpoint_callback(
    "./saved_files/finalUlit",
    "final",
    "val_loss",
    True,
    True
  )
except Exception as e:
  print(f"Error: {e}")
  _helper.notifications.error()

In [20]:
try:
  # here where you will use the models that are saved by tf
  # for this code it used EfficientNetB0 and removing the top to replace with a specified variables for our use case
  model_parts = tf.keras.applications.EfficientNetB0(include_top = False)

  # it sets the model to freeze mode which means that the values inside the nodes wont change but will still learn accordingly to how they are trained
  model_parts.trainable = False

  # this will replace the remove early on the use of finished model that states like this:
  # it is Input that will set the images(shape) into 224x224 with RGB(3) and has a name of layer (name) input_layer
  input = Input(shape = (500, 500, 3), name = "input_layer")

  # here you will connect the input_layer and the model
  x = model_parts(input)

  # here you will now add some layer below / at the end of the process a layer and read like this:
  # adding GlobalAveragePooling2D with a name (name) of GAP2D_layer below the model (x) and stored in x
  x = GlobalAveragePooling2D(name = "GAP2D_layer")(x)

  # adding the final layer which decides which is which and read like this:
  # adding Dense with 3 class, with activation of softmax that is named output_layer below x and stored in output
  output = Dense(3, activation = "softmax", name = "output_layer")(x)

  # now building the model
  # Model has head of input and ends with output that is named "EffNet0_FE"
    ## you can change the name on how you want
  model_test = Model(input, output, name = "Final_Model")

  # compiling the model saying that it is categorical by categorical_crossentropy, has Adam as its optimizer, and will also add information about categoricalaccuracy and precision
  model_test.compile(
    loss = "categorical_crossentropy",
    optimizer = Adam(0.0001),
    metrics = ["CategoricalAccuracy", "Precision"]
  )

  # saving the data of the model into model_test_h for history data while fitting (fit)
  # inside the fit you need train_data(which stores the directory of training sets prepared at the top)
  # - epochs which says how many times do i need to train the model
  # >> note the other are just extras
  # validation_data which need the test_data(which stores the directory of test sets prepared at the top)
  # - to validate it while being trained
  # - callbacks which 1. checkpoint function which is also set at the top, will do checkpointing
  # 2. tensorboard_callback to create a graph of data in ./log/EffNet0 with the name of Model 2
  # >> note you should always create different folder for different models
  # >> note or you should always rename the name when testing with same model but different configuration
  # >> note: also to be aware if you can see the validation_data has the test_data which smells fishy you correct that is improper
  # - because we should have validation dataset to use here and use the test_data for the testing.
  # pero dahil kulang pa tayo data yun muna pansamantala
  model_test_h = model_test.fit(train_data,
                          epochs = 20,
                          validation_data = test_data,
                          callbacks = [checkpoint,
                                      _helper.callbacks.tensorboard_callback("./log/finalUlit",
                                                                              "Final_model")])
  _helper.notifications.model_callback_notification()
except Exception as e:
  print(f"Error: {e}")
  _helper.notifications.error()

Saving TensorBoard Log Files to: ./log/finalUlit/Final_model/2022_10_31__19_07_log
Epoch 1/20
29/29 [==============================] - 250s 8s/step - loss: 1.1357 - categorical_accuracy: 0.4272 - precision: 0.3475 - val_loss: 1.0970 - val_categorical_accuracy: 0.3444 - val_precision: 0.4000
Epoch 2/20
29/29 [==============================] - 236s 8s/step - loss: 0.9760 - categorical_accuracy: 0.5620 - precision: 0.7375 - val_loss: 1.0193 - val_categorical_accuracy: 0.4667 - val_precision: 0.5000
Epoch 3/20
29/29 [==============================] - 224s 8s/step - loss: 0.8580 - categorical_accuracy: 0.7054 - precision: 0.8546 - val_loss: 0.9664 - val_categorical_accuracy: 0.5333 - val_precision: 0.5882
Epoch 4/20
29/29 [==============================] - 210s 7s/step - loss: 0.7723 - categorical_accuracy: 0.7554 - precision: 0.8818 - val_loss: 0.9293 - val_categorical_accuracy: 0.5556 - val_precision: 0.6250
Epoch 5/20
29/29 [==============================] - 224s 8s/step - loss: 0.7081 -

In [21]:
print(ckpt_dir)

./saved_files/finalUlit/checkpoint/final.ckpt


In [22]:
model_test.load_weights(ckpt_dir)

In [23]:
# this is used to evaluate din
model_test.evaluate(test_data)

3/3 [==============================] - 24s 7s/step - loss: 0.7901 - categorical_accuracy: 0.6444 - precision: 0.6538


[0.7901092767715454, 0.644444465637207, 0.6538461446762085]

In [24]:
try:
  # this is used to save the model and/or history
  _helper.save_load.save(model = model_test,
                        path = "./saved_files/h5/finalUlit",
                        name = "Final_1")
except Exception as e:
  print(f"Error {e}")
  _helper.notifications.error()

Model is saved to ./saved_files/h5/finalUlit/Final_1_2022_10_31 - 20.h5 and ./saved_files/h5/finalUlit/Final_1_diagram.png


In [25]:
try: 
  # this is used to save the model to JS
  tfjs.converters.save_keras_model(model_test, "./saved_files/JS/finalUlit/")
except Exception as e:
  print(f"Error {e}")
  _helper.notifications.error()

weight normalization_3/count with shape () and dtype int64 was auto converted to the type int32
